In [2]:
import numpy as np

In [31]:
class GridWorld:
    def __init__(self, taille, position_start, good_end_position, bad_end_position):
        self.current_state = position_start  # État actuel (ligne, colonne)
        self.states = [[x, y] for x in range(taille[0]) for y in range(taille[1])]
        self.end_good_state = good_end_position  # État final (ligne, colonne)
        self.end_bad_state = bad_end_position
        self.grid_size = taille  # Taille de la grille (lignes, colonnes)
        self.stateSpace = {}
        self.matchStates()
        self.currentIntState = self.getStateInt(self.current_state)
        self.num_actions = 4  # Nombre total d'actions possibles (haut, bas, gauche, droite)
        self.reward = 0  # Récompense actuelle
        self.done = False  # Indique si la partie est terminée
        self.generate_grid()
        self.actions = [0, 1, 2, 3]
        self.rewards = [0, 1, 3]
        self.actionSpace = {0: -self.grid_size[0], 1: self.grid_size[0],
                            2: -1, 3: 1}

    
    def reset(self, position_start):
        self.current_state = position_start
        self.currentIntState = self.getStateInt(position_start)
        
    def matchStates(self):
        i=0
        for s in self.states:
            self.stateSpace[str(s)] = i
            i = i+1
    
    def getStateInt(self, st):
        return self.stateSpace[str(st)]
    
    def getStateCouple(self, st):
        n_state = {i for i in self.stateSpace if self.stateSpace[i]==st}
        return list(n_state)
        
    
    def step(self, action):
        if action == 0:
            if self.current_state[0] == 0 :
                self.current_state[0] = self.grid_size[0] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour traverser le mur
                #self.generate_grid()
                self.endgame()
            else:
                self.current_state[0] = self.current_state[0] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()

        elif action == 1:
            if self.current_state[0] == self.grid_size[0] - 1:
                self.current_state[0] = 0
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
            else :
                self.current_state[0] = self.current_state[0] + 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()

        elif action == 2:
            if self.current_state[1] == 0:
                self.current_state[1] = self.grid_size[1] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
            else:
                self.current_state[1] = self.current_state[1] - 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()

        elif action == 3:
            if self.current_state[1] == self.grid_size[1] - 1:
                self.current_state[1] = 0
                self.currentIntState = self.getStateInt(self.current_state)
                #print(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
            else:
                self.current_state[1] = self.current_state[1] + 1
                self.currentIntState = self.getStateInt(self.current_state)
                self.reward = -1  # Pas de récompense pour avancer
                #self.generate_grid()
                self.endgame()
                # Si l'on atteint l'état final, la partie est terminée
        return self.current_state, self.reward, self.done

    def endgame(self):
        if self.current_state == self.end_good_state:
            self.reward = 10  # Récompense de 1 pour atteindre l'état final
            self.done = True
        elif self.current_state == self.end_bad_state:
            self.reward = -10
            self.done = True

    # def update_grid(self):
    #     new_grid = [["_", "_", "_", "_"],
    #                 ["_", "_", "_", "_"],
    #                 ["_", "_", "_", "_"],
    #                 ["_", "_", "_", "_"]]
    #     new_grid[self.current_state[0]][self.current_state[1]] = "X"
    #     for i in new_grid:
    #         print(i)

    def generate_grid(self):
        grid=[]
        for i in range(self.grid_size[0]):
            grid.append([])
            for j in range(self.grid_size[1]):
                grid[i].append("_")
        grid[self.current_state[0]][self.current_state[1]] = "X"
        for i in grid:
            print(i)
        print("\n")
    

In [26]:
def initP(grid):
    P = {}
    for state in grid.states:
        st = grid.getStateInt(state)
        for action in grid.actions:
            grid.current_state = state
            state_, reward_, done_ = grid.step(action)
            stt = grid.getStateInt(state_)
            P[(stt, reward_, st, action)] = 1
    return P

In [27]:
def evaluate_policy(grid, V, policy, discount_factor=0.9):
    theta=1e-6
    P = initP(grid)
    #Initialize the Value function 
    #V = np.zeros((world.grid_size[0],world.grid_size[1]))
    
    while True:
        DELTA = 0
        for s in grid.states:
            i = s[0]
            j = s[1]
            m_state = grid.getStateInt(s)
            old_V = V[i][j]
            weight = 1 / len(policy[m_state])
            for action in policy[m_state]:
                total = 0
                for key in P:
                    (newState, reward, oldState, act) = key
                    n_state = list(grid.getStateCouple(newState)[0])
                    n_state = [int(n_state[1]), int(n_state[4])]
                    if oldState == m_state and act == action:
                        total += weight*P[key]*(reward+discount_factor*V[n_state[0]][n_state[1]])
            V[i][j] = total
            DELTA = max(DELTA, np.abs(old_V-V[i][j]))
            
        if DELTA < theta:
            return V            

        
    

In [28]:
def policy_improvement(grid, V, policy, discount_factor=0.9):
    policy_stable = True
    newPolicy = {}
    for s in grid.states:
        i = s[0]
        j = s[0]
        m_state = grid.getStateInt(s)
        old_actions = policy[m_state]
        value = []
        newAction = []
        P = initP(grid)
        for action in old_actions:
            total = 0
            weight = 1 / len(policy[m_state])
            for key in P:
                (newState, reward, oldState, act) = key
                n_state = list(grid.getStateCouple(newState)[0])
                n_state = [int(n_state[1]), int(n_state[4])]
                if oldState == m_state and act == action:
                    total += weight*P[key]*(reward+discount_factor*V[n_state[0]][n_state[1]])
                    value.append(np.round(total, 2))
                    newAction.append(action)  
                    
        value = np.array(value) #Get the list of gotten value actions in the current state
        print(value)
        best = np.where(value == value.max())[0] #Get the position of the best gotten value action
        bestActions = [newAction[item] for item in best]
        newPolicy[m_state] = bestActions
        
        if old_actions != bestActions:
            policy_stable = False
    return policy_stable, newPolicy

In [29]:
def policy_iteration(grid, discount_factor=0.9):
    
    #Initialize the policy
    V = np.random.random((world.grid_size[0],world.grid_size[1]))
    policy = {}
    for state in grid.stateSpace.values():
        # equiprobable random strategy
        policy[state] = grid.actions
    print("Policy : ")
    print(policy)
    V = evaluate_policy(grid, V, policy, discount_factor=0.9)
    print("V = ")
    print(V)
    
    policy_stable = False
    while not policy_stable:
        old_policy = policy.copy()
        
        print("Start of policy evaluation")
        #Evaluate the policy
        V = evaluate_policy(grid, V, policy, discount_factor=0.9)
        print("V = ")
        print(V)
        
        print("Start of policy Improvement")
        #Improve the policy
        policy_stable, policy =  policy_improvement(grid, V, policy, discount_factor=0.9)
        print("Policy : ")
        print(policy)
    
        """if all(old_policy[s] == policy[s] for s in grid.stateSpace.values()):
            break"""
            
    return policy
 

In [32]:
if __name__ == '__main__':
    # Il faut choisir un nombre entre 0 et 3
    # Si l'action est 0, déplacement vers le haut
    # Si l'action est 1, déplacement vers le bas
    # Si l'action est 2, déplacement vers la gauche
    # Si l'action est 3, déplacement vers la droite
    world = GridWorld([4,4],[0,0],[3,3],[2,3]) #taille, position de départ, fin de jeu reward positive, fin de jeu reward négative
    policy_iteration(world, discount_factor=0.9)
    




['X', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']
['_', '_', '_', '_']


Policy : 
{0: [0, 1, 2, 3], 1: [0, 1, 2, 3], 2: [0, 1, 2, 3], 3: [0, 1, 2, 3], 4: [0, 1, 2, 3], 5: [0, 1, 2, 3], 6: [0, 1, 2, 3], 7: [0, 1, 2, 3], 8: [0, 1, 2, 3], 9: [0, 1, 2, 3], 10: [0, 1, 2, 3], 11: [0, 1, 2, 3], 12: [0, 1, 2, 3], 13: [0, 1, 2, 3], 14: [0, 1, 2, 3], 15: [0, 1, 2, 3]}
V = 
[[-0.32258059 -0.32258062 -0.3225806  -0.32258058]
 [-0.32258056 -0.32258062 -0.32258055 -0.32258056]
 [-0.32258057 -0.32258061 -0.32258055 -3.2258062 ]
 [-0.32258059 -0.32258056 -0.32258056  3.22580627]]
Start of policy evaluation
V = 
[[-0.32258063 -0.32258064 -0.32258063 -0.32258063]
 [-0.32258063 -0.32258064 -0.32258062 -0.32258063]
 [-0.32258063 -0.32258064 -0.32258062 -3.22580639]
 [-0.32258063 -0.32258063 -0.32258063  3.22580641]]
Start of policy Improvement
[-0.32 -0.32 -0.32 -0.32]
[-0.32 -0.32 -0.32 -0.32]
[-0.32 -0.32 -0.32 -0.32]
[ 3.23 -0.32 -0.32 -0.32]
[-0.32 -0.32 -0.32 -0.32]
[-0.32 -0.32 -0.32 -

In [5]:
%sql

UsageError: Line magic function `%sql` not found.
